## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2026-01-06-08-55-40 +0000,bbc,Next reports Christmas rush but warns of slowd...,https://www.bbc.com/news/articles/cre27x179zpo...
1,2026-01-06-08-55-00 +0000,wsj,French Inflation Fell Slightly in December,https://www.wsj.com/economy/central-banking/fr...
2,2026-01-06-08-54-55 +0000,nyt,"At U.N. Emergency Meeting, U.S. Allies Critici...",https://www.nytimes.com/live/2026/01/05/world/...
3,2026-01-06-08-47-42 +0000,nyt,The C.I.A. assessed Venezuela’s future without...,https://www.nytimes.com/live/2026/01/05/world/...
4,2026-01-06-08-46-56 +0000,nyt,Stephen Miller Asserts U.S. Has Right to Take ...,https://www.nytimes.com/2026/01/05/us/politics...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2371/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
18,venezuela,65
22,maduro,56
80,trump,50
86,capture,20
385,oil,18


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
166,2026-01-05-21-27-39 +0000,cbc,"Venezuela's collapse, Trump’s military buildup...",https://www.cbc.ca/news/world/trump-venezuela-...,205
314,2026-01-05-10-00-26 +0000,wapo,"What to know about Maduro, his capture and Tru...",https://www.washingtonpost.com/world/2026/01/0...,197
317,2026-01-05-09-20-28 +0000,wapo,Trump says U.S. is ‘in charge’ in Venezuela af...,https://www.washingtonpost.com/world/2026/01/0...,192
201,2026-01-05-19-32-24 +0000,nyt,"Who Is Delcy Rodríguez, Venezuela’s New Interi...",https://www.nytimes.com/2026/01/05/world/ameri...,185
302,2026-01-05-10-47-00 +0000,wsj,Oil Prices Fall as U.S. Capture of Venezuela’s...,https://www.wsj.com/business/energy-oil/asian-...,176


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
166,205,2026-01-05-21-27-39 +0000,cbc,"Venezuela's collapse, Trump’s military buildup...",https://www.cbc.ca/news/world/trump-venezuela-...
67,71,2026-01-06-02-24-00 +0000,wsj,In the wake of the Trump administration cuttin...,https://www.wsj.com/politics/policy/trump-admi...
186,56,2026-01-05-20-18-05 +0000,nypost,Will Amy Klobuchar run for Minnesota governor?...,https://nypost.com/2026/01/05/us-news/all-eyes...
190,52,2026-01-05-20-12-00 +0000,wsj,Oil Rises As Market Digests Venezuelan Preside...,https://www.wsj.com/finance/commodities-future...
263,47,2026-01-05-15-25-42 +0000,nypost,Paris court finds 10 people guilty of cyberbul...,https://nypost.com/2026/01/05/world-news/paris...
80,42,2026-01-06-01-37-29 +0000,nypost,Newsom claims he was ‘physically’ in middle of...,https://nypost.com/2026/01/05/us-news/newsom-b...
106,38,2026-01-05-23-50-43 +0000,nypost,Iran’s Ayatollah Ali Khamenei plans to flee to...,https://nypost.com/2026/01/05/world-news/irans...
109,33,2026-01-05-23-38-20 +0000,nypost,Nvidia CEO Jensen Huang calls AI titan’s lates...,https://nypost.com/2026/01/05/business/nvidia-...
88,32,2026-01-06-00-57-00 +0000,wsj,The Pentagon is moving to demote Sen. Mark Kel...,https://www.wsj.com/politics/hegseth-moves-to-...
87,31,2026-01-06-00-59-36 +0000,nypost,Somali ambassador to UN is linked to shady Ohi...,https://nypost.com/2026/01/05/us-news/somali-a...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
